# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
#Import required libraries
from bs4 import BeautifulSoup
import requests
import csv
import json
from pandas.io.json import json_normalize
import xml
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

### Download and explore dataset

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

result = BeautifulSoup(source, 'lxml')#Beautiful Soup to Parse the url page

table=result.find('table') #Finds the required table area

In [3]:
#table #Uncomment to view the table

In [4]:
#List initialization to collect the Postalcodes, Boroughs and Neighborhoods
postalcode=[]
borough=[]
neighborhood=[]

x=table.tbody #Navigation to the body of the table

for tr in x.find_all('tr'): #Iterating through rows 
    td=tr.find_all('td')
    counter = 1
    postalcode_value = -1
    borough_value = -1
    neighborhood_value = -1
    for cell in td: #Itering through columns
        if counter == 1:
            postalcode_value = cell.text #Collecting the Postalcodes
        if counter == 2:
            borough_value = cell.text
            tag_a_borough = cell.find('a')
        if counter == 3:
            neighborhood_value = str(cell.text).strip()
            tag_a_neighborhood = cell.find('a')
        
        counter +=1
    #Exclude postal codes with 'Not Assigned' values for Borough or Neighborhood 
    if (postalcode_value == 'Not Assigned' or borough_value == 'Not Assigned' or neighborhood_value == 'Not Assigned'):
        continue
    try:
        if ((tag_a_borough is None) or (tag_a_neighborhood is None)):
            continue
    except:
        pass
    if (postalcode_value == -1 or borough_value == -1 or neighborhood_value == -1):
        continue
#Append derived values to the list        
    postalcode.append(postalcode_value)
    borough.append(borough_value)
    neighborhood.append(neighborhood_value)

In [5]:
unique_p = set(postalcode)
print('num of unique Postal codes:', len(unique_p))
postalcode_u      = []
borough_u       = []
neighborhood_u = []


for postalcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postalcode_idx, postalcode_element in enumerate(postalcode):
        if postalcode_unique_element == postalcode_element:
            p_var = postalcode_element;
            b_var = borough[postalcode_idx]
            if n_var == '': 
                n_var = neighborhood[postalcode_idx]
            else:
                n_var = n_var + ', ' + neighborhood[postalcode_idx]
    postalcode_u.append(p_var)
    borough_u.append(b_var)
    neighborhood_u.append(n_var)

num of unique Postal codes: 84


### Transform data into Pandas DataFrame

In [6]:
df_toronto = pd.DataFrame({'PostalCode':postalcode_u,'Borough':borough_u,'Neighborhood':neighborhood_u})
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9W,Etobicoke,Northwest
4,M9L,North York,Humber Summit


In [7]:
df_toronto.shape

(84, 3)